# Sprint ディープラーニングフレームワーク1

### コードリーディング
TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。<br>
<br>
tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。<br>
<br>
（例）<br>
- 重みを初期化する必要があった
- エポックのループが必要だった<br>


それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。<br>

### データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。<br>
<br>
[Iris Species](https://www.kaggle.com/uciml/iris/data)<br>
<br>
目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。
- Iris-versicolor
- Iris-virginica


>ディープラーニングに必要な項目の一例：
>- レイヤーの関数及び結合方法の選択
>- 最適化手法・パラメータ初期化法の選択
>- レイヤーのノード数を計算
>- ミニバッチ取得
>- 逆伝播によるパラメータ更新

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。<br>
<br>
それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [1]:
# 《サンプルコード》
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Colocations handled automatically by placer.
Epoch 0, loss : 43.6994, val_loss : 16.3642, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 10.5538, val_loss : 21.7286, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 7.8144, val_loss : 3.5118, acc : 0.500, val_acc : 0.688
Epoch 3, loss : 1.4844, val_loss : 3.4646, acc : 0.750, val_acc : 0.625
Epoch 4, loss : 0.5620, val_loss : 0.8597, acc : 0.750, val_acc : 0.750
Epoch 5, loss : 0.0001, val_loss : 0.9200, acc : 1.000, val_acc : 0.875
Epoch 6, loss : 0.0000, val_loss : 0.0099, acc : 1.000, val_acc : 1.000
Epoch 7, loss : 0.0000, val_loss : 0.0029, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.0000, val_loss : 0.0042, acc : 1.000, val_acc : 1.000
test_acc : 0.900


>- レイヤーの関数及び結合方法の選択　→　レイヤーをdefine_netで宣言し、データを入れた状態をlogitsに代入。logitsを目的関数で回す。
>- 最適化手法・パラメータ初期化法の選択　→ 最適化手法をoptimizerインスタンスとして実装し、minimizeメソッドで損失を最小化。パラメータはtf.random_normalで初期化
>- レイヤーのノード数を計算　→ ノード数はtf.random_normalで指定可能
>- ミニバッチ取得　→ ミニバッチ取得関数を実装
>- 逆伝播によるパラメータ更新　→ sessionのrun関数よりloss_opを呼びだす。更新はrun内で行われる

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。<br>
<br>
[Iris Species](https://www.kaggle.com/uciml/iris/data)<br>
<br>
2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。<br>
<br>
<br>
**《ヒント》**<br>
<br>
以下の2箇所は2クラス分類特有の処理です。

```
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
```

メソッドは以下のように公式ドキュメントを確認してください。<br>
<br>
[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)<br>
<br>
[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)<br>
<br>
＊tf.signとtf.math.signは同じ働きをします。

In [82]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを多値分類する
"""
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルをone-hotに変換
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis]).astype(np.int)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits), axis=0)
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
# correct_pred = tf.equal(Y, tf.nn.softmax(logits))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 132.0380, val_loss : 65.3910, acc : 0.444, val_acc : 0.833
Epoch 1, loss : 13.3728, val_loss : 2.6306, acc : 0.556, val_acc : 0.972
Epoch 2, loss : 0.0718, val_loss : 96.2536, acc : 1.000, val_acc : 0.750
Epoch 3, loss : 0.0498, val_loss : 142.2392, acc : 1.000, val_acc : 0.750
Epoch 4, loss : 0.0001, val_loss : 124.0513, acc : 1.000, val_acc : 0.778
Epoch 5, loss : 0.0000, val_loss : 42.1953, acc : 1.000, val_acc : 0.833
Epoch 6, loss : 0.0008, val_loss : 28.0241, acc : 1.000, val_acc : 0.944
Epoch 7, loss : 0.0001, val_loss : 26.8227, acc : 1.000, val_acc : 0.944
Epoch 8, loss : 0.0034, val_loss : 36.2017, acc : 1.000, val_acc : 0.944
Epoch 9, loss : 0.0000, val_loss : 0.4353, acc : 1.000, val_acc : 1.000
test_acc : 0.978


## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。<br>
<br>
[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)<br>
<br>
この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。<br>
<br>
分類問題と回帰問題の違いを考慮してください。

In [2]:
from sklearn.preprocessing import StandardScaler

house_data = pd.read_csv('/Users/tamiyagt/Documents/machine learning/02_Kaggle/house prices/train.csv')

#GrLivArea、YearBuilt、SalePriceを抽出
train = house_data.loc[:, ['GrLivArea', 'YearBuilt', 'SalePrice']]

# DataFrameをndarrayに変換
X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])[:,None]

# 特徴量を標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)
# 目的変数を対数変換
y = np.log1p(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [3]:
"""
TensorFlowで実装したニューラルネットワークを使いHousePricesデータセットを多値分類する
"""
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.squared_difference(Y, logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果（不要）
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.squared_difference(Y, logits))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, MSE : {:.3f}, val_MSE : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_MSE : {:.3f}".format(test_acc))

Epoch 0, loss : 5.5262, val_loss : 8.9015, MSE : 5.526, val_MSE : 8.902
Epoch 1, loss : 0.3059, val_loss : 3.8463, MSE : 0.306, val_MSE : 3.846
Epoch 2, loss : 0.4133, val_loss : 2.5670, MSE : 0.413, val_MSE : 2.567
Epoch 3, loss : 0.4706, val_loss : 2.0837, MSE : 0.471, val_MSE : 2.084
Epoch 4, loss : 0.3683, val_loss : 1.6049, MSE : 0.368, val_MSE : 1.605
Epoch 5, loss : 0.3062, val_loss : 1.1680, MSE : 0.306, val_MSE : 1.168
Epoch 6, loss : 0.2007, val_loss : 0.9513, MSE : 0.201, val_MSE : 0.951
Epoch 7, loss : 0.1601, val_loss : 0.8620, MSE : 0.160, val_MSE : 0.862
Epoch 8, loss : 0.1321, val_loss : 0.7652, MSE : 0.132, val_MSE : 0.765
Epoch 9, loss : 0.0993, val_loss : 0.6455, MSE : 0.099, val_MSE : 0.646
test_MSE : 0.852


## 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。<br>
<br>
3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。<br>
<br>
スクラッチで実装したモデルの再現を目指してください。

In [109]:
#《データセットをダウンロードするコード》
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

X_train = X_train[:,:,:,None]  # NHWC
X_test = X_test[:,:,:,None]  # NHWC

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])

# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [136]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTデータセットを多値分類する
"""

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
stride = 1
pad = 'VALID'
ksize = [3, 3]
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, X_train.shape[1], X_train.shape[2], X_train.shape[3]])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([5,5,1,4])),  # HWCF
        'w2': tf.Variable(tf.random_normal([3,3,4,16])),
        'w3': tf.Variable(tf.random_normal([64, 32])),
        'w4': tf.Variable(tf.random_normal([32, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([1,1,1,4])),
        'b2': tf.Variable(tf.random_normal([1,1,1,16])),
        'b3': tf.Variable(tf.random_normal([32])),
        'b4': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.nn.conv2d(x, weights['w1'], stride, pad) + biases['b1']
    layer_1 = tf.nn.max_pool2d(layer_1, ksize, ksize, pad)
    layer_2 = tf.nn.conv2d(layer_1, weights['w2'], stride, pad) + biases['b2']
    layer_2 = tf.nn.max_pool2d(layer_2, ksize, ksize, pad)
    layer_2 = tf.layers.Flatten()(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_output = tf.matmul(layer_3, weights['w4']) + biases['b4'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits), axis=0)
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    print(sess.run(tf.argmax(tf.nn.softmax(logits), 1), feed_dict={X: X_val, Y: y_val}))

Epoch 0, loss : 1.4045, val_loss : 0.5528, acc : 0.700, val_acc : 0.889
Epoch 1, loss : 0.3883, val_loss : 0.2519, acc : 0.900, val_acc : 0.932
Epoch 2, loss : 0.0093, val_loss : 0.2393, acc : 1.000, val_acc : 0.938
Epoch 3, loss : 0.1637, val_loss : 0.3166, acc : 0.900, val_acc : 0.927
Epoch 4, loss : 0.0018, val_loss : 0.2079, acc : 1.000, val_acc : 0.952
Epoch 5, loss : 0.0025, val_loss : 0.2371, acc : 1.000, val_acc : 0.949
Epoch 6, loss : 0.0525, val_loss : 0.1717, acc : 1.000, val_acc : 0.960
Epoch 7, loss : 0.0008, val_loss : 0.1481, acc : 1.000, val_acc : 0.966
Epoch 8, loss : 0.1534, val_loss : 0.2951, acc : 0.800, val_acc : 0.943
Epoch 9, loss : 0.9862, val_loss : 0.2019, acc : 0.900, val_acc : 0.957
test_acc : 0.964
[3 6 6 ... 5 1 6]


In [129]:
np.argmax(y_test, axis=1)

array([7, 2, 1, ..., 4, 5, 6])